In [64]:
import pandas as pd

df_train = pd.read_csv('train.txt', header =None, sep =';', names = ['Input','Sentiment'], encoding='utf-8')
df_test = pd.read_csv('test.txt', header = None, sep =';', names = ['Input','Sentiment'],encoding='utf-8')

In [65]:
print(df_train.head())
print(df_test.head())

                                               Input Sentiment
0                            i didnt feel humiliated   sadness
1  i can go from feeling so hopeless to so damned...   sadness
2   im grabbing a minute to post i feel greedy wrong     anger
3  i am ever feeling nostalgic about the fireplac...      love
4                               i am feeling grouchy     anger
                                               Input Sentiment
0  im feeling rather rotten so im not very ambiti...   sadness
1          im updating my blog because i feel shitty   sadness
2  i never make her separate from me because i do...   sadness
3  i left with my bouquet of red and yellow tulip...       joy
4    i was feeling a little vain when i did this one   sadness


In [66]:
# Number of unique classes in the Sentiment column of df_test
num_unique_classes_train = df_train['Sentiment'].nunique()
print("Number of unique classes in df_test Sentiment column:", num_unique_classes_train)
# Number of unique classes in the Sentiment column of df_test
num_unique_classes_test = df_test['Sentiment'].nunique()
print("Number of unique classes in df_test Sentiment column:", num_unique_classes_test)

Number of unique classes in df_test Sentiment column: 6
Number of unique classes in df_test Sentiment column: 6


In [67]:
# Get unique classes in the Sentiment column of df_test
unique_classes_train = df_train['Sentiment'].unique()
print("Unique classes in df_train Sentiment column:", unique_classes_train)
# Get unique classes in the Sentiment column of df_test
unique_classes_test = df_test['Sentiment'].unique()
print("Unique classes in df_test Sentiment column:", unique_classes_test)

Unique classes in df_train Sentiment column: ['sadness' 'anger' 'love' 'surprise' 'fear' 'joy']
Unique classes in df_test Sentiment column: ['sadness' 'joy' 'fear' 'anger' 'love' 'surprise']


In [68]:
encoded_dict = {'anger':0,'fear':1, 'joy':2, 'love':3, 'sadness':4, 'surprise':5}
df_train['Sentiment'] = df_train.Sentiment.map(encoded_dict)
df_test['Sentiment'] = df_test.Sentiment.map(encoded_dict)

In [69]:
df_test.head()

,Input,Sentiment
0,im feeling rather rotten so im not very ambiti...,4
1,im updating my blog because i feel shitty,4
2,i never make her separate from me because i do...,4
3,i left with my bouquet of red and yellow tulip...,2
4,i was feeling a little vain when i did this one,4


In [70]:
# Check for any unmapped values or NaNs
print(df_train['Sentiment'].isnull().sum(), df_test['Sentiment'].isnull().sum())

0 0


In [71]:
import tensorflow
from tensorflow.keras.utils import to_categorical
#converting our integer coded Sentiment column into categorical data(matrix)
y_train = to_categorical(df_train.Sentiment)
y_test = to_categorical(df_test.Sentiment)

In [72]:
#!pip install tensorflow

In [73]:
# !pip install transformers

In [74]:
import transformers

In [75]:
from transformers import AutoTokenizer,TFBertModel
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
bert = TFBertModel.from_pretrained('bert-base-cased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [76]:
# !pip install tf-keras

In [77]:
'''
We need a tokenizer to convert the input text’s word into tokens.

The classAutoTokenizer contains various types of tokenizers.

TFBertModel pre-trained Bert model for TensorFlow.
'''

'\nWe need a tokenizer to convert the input text’s word into tokens.\n\nThe classAutoTokenizer contains various types of tokenizers.\n\nTFBertModel pre-trained Bert model for TensorFlow.\n'

In [78]:
# Tokenize the input (takes some time) 
# here tokenizer using from bert-base-cased
x_train = tokenizer(
    text=df_train.Input.tolist(),
    add_special_tokens=True,
    max_length=70,
    truncation=True,
    padding='max_length', 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)
x_test = tokenizer(
    text=df_test.Input.tolist(),
    add_special_tokens=True,
    max_length=70,
    truncation=True,
    padding='max_length', 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

In [79]:
input_ids = x_train['input_ids']
attention_mask = x_train['attention_mask']

In [80]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense, Lambda, GlobalMaxPool1D, Dropout

In [81]:
# Define model inputs
max_len = 70
input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
attention_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")

# Define the BERT embedding layer
def get_bert_embeddings(inputs):
    input_ids, attention_mask = inputs
    return bert(input_ids, attention_mask=attention_mask)[0]

# Use Lambda layer to wrap the BERT model
embeddings = Lambda(get_bert_embeddings, output_shape=(max_len, 768), name='BERT_Embeddings')([input_ids, attention_mask])

# Define the rest of the model
out = GlobalMaxPool1D()(embeddings)
out1 = Dense(128, activation='relu')(out)
out2 = Dropout(0.1)(out1)
out3 = Dense(32, activation='relu')(out2)
y = Dense(6, activation='sigmoid')(out3)

# Create the model
model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=y)
model.layers[2].trainable = True

In [82]:
optimizer = Adam(
    learning_rate=5e-05, 
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0)
# Set loss and metrics
loss =CategoricalCrossentropy(from_logits = True)
metric = CategoricalAccuracy('balanced_accuracy')
# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = [metric])

C:\Users\Abhishek\anaconda3\Lib\site-packages\keras\src\optimizers\base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


In [83]:
# Define a custom callback to log input shapes
class LogInputShapesCallback(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        print(f"Epoch {epoch+1} - Input shape: {self.model.input}")

In [84]:
train_history = model.fit(
    x ={'input_ids':x_train['input_ids'],'attention_mask':x_train['attention_mask']} ,
    y = y_train,
    validation_data = (
    {'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']}, y_test
    ),
    epochs=10,
    batch_size=36,
    callbacks=[LogInputShapesCallback()]  # Add the custom callback here
)

Epoch 1 - Input shape: [<KerasTensor shape=(None, 70), dtype=int32, sparse=None, name=input_ids>, <KerasTensor shape=(None, 70), dtype=int32, sparse=None, name=attention_mask>]
Epoch 1/10


C:\Users\Abhishek\anaconda3\Lib\site-packages\keras\src\backend\tensorflow\nn.py:560: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


445/445 ━━━━━━━━━━━━━━━━━━━━ 2368s 5s/step - balanced_accuracy: 0.3386 - loss: 1.6535 - val_balanced_accuracy: 0.4540 - val_loss: 1.4621
Epoch 2 - Input shape: [<KerasTensor shape=(None, 70), dtype=int32, sparse=None, name=input_ids>, <KerasTensor shape=(None, 70), dtype=int32, sparse=None, name=attention_mask>]
Epoch 2/10
445/445 ━━━━━━━━━━━━━━━━━━━━ 2512s 6s/step - balanced_accuracy: 0.4403 - loss: 1.4990 - val_balanced_accuracy: 0.4980 - val_loss: 1.3924
Epoch 3 - Input shape: [<KerasTensor shape=(None, 70), dtype=int32, sparse=None, name=input_ids>, <KerasTensor shape=(None, 70), dtype=int32, sparse=None, name=attention_mask>]
Epoch 3/10
445/445 ━━━━━━━━━━━━━━━━━━━━ 2447s 6s/step - balanced_accuracy: 0.4878 - loss: 1.4116 - val_balanced_accuracy: 0.5090 - val_loss: 1.3435
Epoch 4 - Input shape: [<KerasTensor shape=(None, 70), dtype=int32, sparse=None, name=input_ids>, <KerasTensor shape=(None, 70), dtype=int32, sparse=None, name=attention_mask>]
Epoch 4/10
445/445 ━━━━━━━━━━━━━━━━━

KeyboardInterrupt: 

In [85]:
predicted_raw = model.predict({'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']})
predicted_raw[0]

63/63 ━━━━━━━━━━━━━━━━━━━━ 263s 4s/step


array([0.5541551 , 0.56640947, 0.8505525 , 0.5073711 , 0.7676348 ,
       0.20156044], dtype=float32)

In [86]:
import numpy as np
y_predicted = np.argmax(predicted_raw, axis = 1)
y_true = df_test.Sentiment

In [87]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_predicted))

              precision    recall  f1-score   support

           0       0.50      0.01      0.02       275
           1       0.55      0.27      0.36       224
           2       0.57      0.90      0.69       695
           3       0.00      0.00      0.00       159
           4       0.52      0.70      0.60       581
           5       0.00      0.00      0.00        66

    accuracy                           0.55      2000
   macro avg       0.36      0.31      0.28      2000
weighted avg       0.48      0.55      0.46      2000



C:\Users\Abhishek\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Abhishek\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Abhishek\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [88]:
texts = input(str('input the text'))
x_val = tokenizer(
    text=texts,
    add_special_tokens=True,
    max_length=70,
    truncation=True,
    padding='max_length', 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True) 
validation = model.predict({'input_ids':x_val['input_ids'],'attention_mask':x_val['attention_mask']})*100
for key , value in zip(encoded_dict.keys(),validation[0]):
    print(key,value)

input the text U bastard


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 365ms/step
anger 68.2198
fear 33.652805
joy 84.7766
love 63.219036
sadness 85.156136
surprise 17.960947
